# Model with residual connections and batch normalization

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Loading train, validation and test sets

In [3]:
ds = tfds.load('malaria', 
               split=('train[:80%]', 'train[80%:]'), 
               shuffle_files=False,
              data_dir='/content/gdrive/MyDrive/datasets/Malaria',
               batch_size=32,
              download=True,
              as_supervised=True,
              with_info=False)

# Model building

In [4]:
def build_model(init_filters, kernel_size, num_resid):
    '''I keep the number of filters the same in all layers in this version'''
    inputs = keras.Input(shape=(None, None, 3))
    x = keras.layers.Rescaling(scale=1.0 / 255)(inputs)
    x = keras.layers.Conv2D(
      filters=init_filters,
      kernel_size=kernel_size,
      strides=(1, 1),
      padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)
    x = keras.layers.MaxPooling2D(
        pool_size=(2, 2), 
        strides=None, 
        padding="same")(x)
    prev_block_output = x
    for i in range(1, num_resid + 1):
        # i is the number of blocks with residual connections
      x = keras.layers.Conv2D(
          filters=init_filters,
          kernel_size=kernel_size,
          strides=(1, 1),
          padding="same")(x)
      x = keras.layers.BatchNormalization()(x)
      x = keras.layers.Activation("relu")(x)
      x = keras.layers.Conv2D(
          filters=init_filters,
          kernel_size=kernel_size,
          strides=(1, 1),
          padding="same")(x)
      x = keras.layers.BatchNormalization()(x)
      x = keras.layers.Activation("relu")(x)
      x = keras.layers.add([x, prev_block_output])
      prev_block_output = x
    x = keras.layers.Conv2D(
        filters=init_filters,
        kernel_size=kernel_size,
        strides=(1, 1),
        padding="same")(prev_block_output)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)
    x = keras.layers.GlobalMaxPooling2D()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=init_filters,
                            activation="relu",
                          kernel_regularizer=None)(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='Adam',
         loss='binary_crossentropy',
         metrics=['accuracy'])    
    return model

In [5]:
model = build_model(64, 4, 6)

In [6]:
history = model.fit(
    ds[0],
    epochs=9,
    verbose="auto",
)

Epoch 1/9
689/689 [==============================] - 561s 786ms/step - loss: 0.1890 - accuracy: 0.9382
Epoch 2/9
689/689 [==============================] - 307s 445ms/step - loss: 0.1433 - accuracy: 0.9553
Epoch 3/9
689/689 [==============================] - 307s 446ms/step - loss: 0.1359 - accuracy: 0.9582
Epoch 4/9
689/689 [==============================] - 306s 444ms/step - loss: 0.1311 - accuracy: 0.9584
Epoch 5/9
689/689 [==============================] - 307s 445ms/step - loss: 0.1301 - accuracy: 0.9594
Epoch 6/9
689/689 [==============================] - 307s 445ms/step - loss: 0.1268 - accuracy: 0.9599
Epoch 7/9
689/689 [==============================] - 307s 445ms/step - loss: 0.1252 - accuracy: 0.9604
Epoch 8/9
689/689 [==============================] - 307s 445ms/step - loss: 0.1221 - accuracy: 0.9609
Epoch 9/9
689/689 [==============================] - 307s 445ms/step - loss: 0.1200 - accuracy: 0.9609


In [7]:
model.evaluate(ds[1])

173/173 [==============================] - 34s 188ms/step - loss: 0.2084 - accuracy: 0.9327


[0.2084464132785797, 0.932692289352417]

#Conclusion
Batch normalization did not improve accuracy. Actually, it is worse than in any of the previous attempts.